## Import required modules


In [ ]:
# Standard import
import os
import numpy as np
import plotly.graph_objects as go

# Move to root directory for easier module import
os.chdir("../../")

# LBAE modules
from modules.maldi_data import MaldiData
from modules.figures import Figures
from modules.atlas import Atlas


# Objects containing our data as well as the atlas data
data = MaldiData()
atlas = Atlas(data, resolution=25)
figures = Figures(data, atlas)


In [ ]:
# Decrease dimensionality because plot is heavy
decrease_dimensionality_factor_root = 8

# get array of annotations, which associate coordinate to id
array_reference_root = np.array(atlas.bg_atlas.reference, dtype=np.int32)

# Subsample array of annotation the same way array_atlas was subsampled
array_reference_root = array_reference_root[
    ::decrease_dimensionality_factor_root,
    ::decrease_dimensionality_factor_root,
    ::decrease_dimensionality_factor_root,
]




In [ ]:
# Crop useless data
def crop_array_to_minimum(array, min_to_consider = 2):
    # Define min and max as image dimensions initially
    x_min, x_max, y_min, y_max, z_min, z_max = (
        0,
        array.shape[0],
        0,
        array.shape[1],
        0,
        array.shape[2],
    )

    # Crop unfilled parts to save space
    for x in range(0, array.shape[0]):
        if np.max(array[x, :, :]) > min_to_consider:
            x_min = max(x_min, x - 1)
            break

    for x in range(array.shape[0] - 1, -1, -1):
        if np.max(array[x, :, :]) > min_to_consider:
            x_max = min(x + 1, x_max)
            break

    for y in range(0, array.shape[1]):
        if np.max(array[:, y, :]) > min_to_consider:
            y_min = max(y - 1, y_min)
            break

    for y in range(array.shape[1] - 1, -1, -1):
        if np.max(array[:, y, :]) > min_to_consider:
            y_max = min(y + 1, y_max)
            break

    for z in range(0, array.shape[2]):
        if np.max(array[:, :, z]) > min_to_consider:
            z_min = max(z - 1, z_min)
            break

    for z in range(array.shape[2] - 1, -1, -1):
        if np.max(array[:, :, z]) > min_to_consider:
            z_max = min(z + 1, z_max)
            break

    # Return the extrema indices
    return x_min, x_max, y_min, y_max, z_min, z_max

# Find out minimum dimensions to keep
x_min, x_max, y_min, y_max, z_min, z_max = crop_array_to_minimum(array_reference_root)


# Crop
array_reference_root = array_reference_root[x_min:x_max, y_min:y_max, z_min:z_max]

#bug correction for the last slice
array_reference_root = np.concatenate(
   (
       array_reference_root,
       np.zeros((1, array_reference_root.shape[1], array_reference_root.shape[2])),
   )
)

In [ ]:
# Define domain for plotting
X_root, Y_root, Z_root = np.mgrid[
    0 : array_reference_root.shape[0]
    / 1000
    * 25
    * decrease_dimensionality_factor_root : array_reference_root.shape[0]
    * 1j,
    0 : array_reference_root.shape[1]
    / 1000
    * 25
    * decrease_dimensionality_factor_root : array_reference_root.shape[1]
    * 1j,
    0 : array_reference_root.shape[2]
    / 1000
    * 25
    * decrease_dimensionality_factor_root : array_reference_root.shape[2]
    * 1j,
]


In [ ]:
# Export plot data (caution, dimensions mixed up for correct orientation)
x_rotating_brain = list(X_root.flatten())
y_rotating_brain = list(Z_root.flatten())
z_rotating_brain = list(-Y_root.flatten())
value_rotating_brain = list(array_reference_root.flatten())

with open('assets/data-rotating-brain.js', 'w') as f:
    f.write('let ' + f'{x_rotating_brain=}' + ';\n')
    f.write('let ' + f'{y_rotating_brain=}' + ';\n')
    f.write('let ' + f'{z_rotating_brain=}' + ';\n')
    f.write('let ' + f'{value_rotating_brain=}' + ';\n')


In [ ]:
# Plot
brain_root_data = go.Volume(
    x=x_rotating_brain,  
    y=y_rotating_brain,
    z=z_rotating_brain,  
    value=value_rotating_brain,
    isomin=9,
    isomax=500,
    opacityscale = "uniform",
    opacity = 0.75,
    surface_count=20,
    colorscale="viridis",  
    flatshading=False,
)


plot_fig = False
if plot_fig:
    fig = go.Figure(data=brain_root_data)

    fig.update_layout(
        margin=dict(t=0, r=0, b=0, l=0),
        scene=dict(
            xaxis=dict(backgroundcolor="rgba(0,0,0,0)", color="grey", gridcolor="grey"),
            yaxis=dict(backgroundcolor="rgba(0,0,0,0)", color="grey", gridcolor="grey"),
            zaxis=dict(backgroundcolor="rgba(0,0,0,0)", color="grey", gridcolor="grey"),
        ),
    )
    fig.show()